Пайплайны sklearn: Pipeline и ColumnTransformer — практикум

**Как работать:**
- Сначала вспоминаем идею пайплайнов, затем сразу собираем свой.
- После каждого шага отвечаем на вопрос «зачем это в продакшене?».
- Примеры из `boostings_101.ipynb` и `eda-feat-engeen.ipynb` можно подсматривать, но код пишем сами.

**Ключевые навыки:**
- Собирать цепочки препроцессинга и модели без утечек.
- Разносить числовые и категориальные признаки через ColumnTransformer.
- Проверять, что пайплайн работает в кросс-валидации и на новых данных.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification, fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


### Практическая траектория
1. Мини-базлайн без пайплайна (только fit/predict).
2. Первый Pipeline с масштабированием и моделью.
3. ColumnTransformer: разделяем разные типы признаков.
4. Валидация: проверяем, что пайплайн не течёт в cross_val_score.
5. Эксперимент: добавьте свой шаг (например, кодирование редких категорий) и оцените эффект.


---
## 1. Зачем нужны пайплайны?

**Проблема без пайплайнов:**

```python
# ❌ Так делать НЕЛЬЗЯ — утечка данных!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # fit на ВСЕХ данных
X_train, X_test = train_test_split(X_scaled, ...)
```

**Проблемы:**
1. Утечка данных (scaler видит test данные)
2. Код не воспроизводим (много ручных шагов)
3. Легко забыть применить transform к новым данным

**Решение — Pipeline:**

```python
# ✅ Так правильно
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])
pipe.fit(X_train, y_train)  # scaler.fit только на train!
pipe.predict(X_test)  # автоматически применяет transform
```


---
## 2. Pipeline: базовый пример


In [ ]:
# Создаём синтетические данные
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=10,
    n_classes=2, random_state=RANDOM_STATE
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")


In [ ]:
# Создаём простой пайплайн
simple_pipe = Pipeline([
    ('scaler', StandardScaler()),           # Шаг 1: стандартизация
    ('classifier', LogisticRegression())    # Шаг 2: модель
])

# Обучаем — scaler.fit_transform на train, потом model.fit
simple_pipe.fit(X_train, y_train)

# Предсказываем — scaler.transform на test, потом model.predict
y_pred = simple_pipe.predict(X_test)
y_pred_proba = simple_pipe.predict_proba(X_test)[:, 1]

print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")


**Альтернативный синтаксис — make_pipeline:**

Автоматически называет шаги по именам классов (в нижнем регистре).


In [ ]:
# То же самое, но короче
simple_pipe_v2 = make_pipeline(StandardScaler(), LogisticRegression())

# Смотрим на шаги
print("Шаги пайплайна:")
for name, step in simple_pipe_v2.named_steps.items():
    print(f"  {name}: {type(step).__name__}")


---
## 3. ColumnTransformer: обработка разных типов признаков

В реальных данных есть числовые и категориальные признаки. Им нужна разная обработка!

**ColumnTransformer** позволяет применять разные трансформации к разным колонкам.


### Практика: соберите первый рабочий Pipeline
Заполните TODO: создайте пайплайн с масштабированием и логистической регрессией.
Обязательно убедитесь, что scaler обучается только на train внутри CV.


In [ ]:
# TODO: ваш первый Pipeline
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# pipeline = Pipeline([
#     ("scaler", StandardScaler()),
#     ("model", LogisticRegression(max_iter=500))
# ])
# scores = cross_val_score(pipeline, X, y, cv=5)
# print("CV accuracy:", scores)


In [ ]:
# Создаём датасет с разными типами признаков
np.random.seed(RANDOM_STATE)
n = 500

df = pd.DataFrame({
    'age': np.random.randint(18, 70, n),
    'income': np.random.randint(20000, 200000, n),
    'credit_score': np.random.randint(300, 850, n),
    'education': np.random.choice(['high_school', 'bachelor', 'master', 'phd'], n),
    'employment': np.random.choice(['employed', 'self_employed', 'unemployed'], n),
    'city': np.random.choice(['Moscow', 'SPb', 'Kazan', 'Novosibirsk', 'Ekaterinburg'], n),
})

# Добавляем пропуски
df.loc[np.random.choice(n, 30, replace=False), 'income'] = np.nan
df.loc[np.random.choice(n, 20, replace=False), 'education'] = np.nan

# Таргет (бинарная классификация)
y = (df['income'].fillna(df['income'].median()) > 80000).astype(int)

print("Пример данных:")
print(df.head(10))
print(f"\nПропуски:\n{df.isnull().sum()}")


In [ ]:
# Определяем типы колонок
numeric_features = ['age', 'income', 'credit_score']
categorical_features = ['education', 'employment', 'city']

print(f"Числовые признаки: {numeric_features}")
print(f"Категориальные признаки: {categorical_features}")


In [ ]:
# Создаём пайплайны для каждого типа признаков

# Для числовых: заполняем пропуски медианой + стандартизация
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Для категориальных: заполняем пропуски модой + OneHot encoding
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Объединяем всё в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print("Preprocessor создан!")


In [ ]:
# Полный пайплайн: предобработка + модель
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=RANDOM_STATE))
])

# Разбиваем данные
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Обучаем
full_pipeline.fit(X_train, y_train)

# Оцениваем
y_pred_proba = full_pipeline.predict_proba(X_test)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")


In [ ]:
# Смотрим, какие фичи получились после препроцессинга
preprocessor.fit(X_train)

# Получаем имена фичей
cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)

all_feature_names = list(numeric_features) + list(cat_feature_names)
print(f"Всего признаков после препроцессинга: {len(all_feature_names)}")
print(f"\nПризнаки: {all_feature_names}")


---
## 4. Обработка категориальных признаков

Существует несколько способов кодирования категорий:

| Метод | Когда использовать | Плюсы | Минусы |
|-------|-------------------|-------|--------|
| **OneHotEncoder** | Мало уникальных значений | Не вносит порядок | Много колонок |
| **OrdinalEncoder** | Есть естественный порядок | Компактно | Вносит ложный порядок |
| **TargetEncoder** | Много категорий, деревья | Компактно, информативно | Риск утечки! |

### 4.1 OneHotEncoder (дефолтный выбор)


### Практика: ColumnTransformer под ваш датасет
- Разделите признаки на числовые и категориальные.
- Добавьте разные преобразования (например, StandardScaler для числовых и OneHotEncoder для категориальных).
- Сравните метрики до/после разделения.


In [ ]:
# TODO: настройте ColumnTransformer и оберните в Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# numeric_features = [...]
# categorical_features = [...]
# preprocessor = ColumnTransformer(
#     [
#         ("num", StandardScaler(), numeric_features),
#         ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
#     ]
# )
# model = ...
# full_pipeline = Pipeline([
#     ("preprocess", preprocessor),
#     ("model", model)
# ])
# scores = cross_val_score(full_pipeline, X, y, cv=5, scoring="roc_auc")
# print(scores)


In [ ]:
# Пример OneHotEncoder
sample_data = pd.DataFrame({
    'color': ['red', 'blue', 'green', 'red', 'blue']
})

ohe = OneHotEncoder(sparse_output=False)
encoded = ohe.fit_transform(sample_data)

print("Исходные данные:")
print(sample_data['color'].values)
print("\nПосле OneHot:")
print(pd.DataFrame(encoded, columns=ohe.get_feature_names_out()))


### 4.2 OrdinalEncoder (для упорядоченных категорий)


In [ ]:
# Пример OrdinalEncoder для упорядоченных категорий
sample_data = pd.DataFrame({
    'size': ['S', 'M', 'L', 'XL', 'M', 'S']
})

# Задаём порядок категорий явно!
oe = OrdinalEncoder(categories=[['S', 'M', 'L', 'XL']])
encoded = oe.fit_transform(sample_data)

print("Исходные данные:")
print(sample_data['size'].values)
print("\nПосле Ordinal (с заданным порядком):")
print(encoded.flatten())


### 4.3 Target Encoding (осторожно — риск утечки!)

**Target Encoding** заменяет категорию на среднее значение таргета для этой категории.

**Проблема:** Если делать на всех данных — утечка! Среднее таргета для категории «подглядывает» в test.

**Безопасный способ:** Использовать `TargetEncoder` из sklearn (версия 1.3+), который автоматически делает CV внутри.


In [ ]:
# Пример Target Encoding (sklearn >= 1.3)
try:
    from sklearn.preprocessing import TargetEncoder
    
    # Данные с высокой кардинальностью
    sample_X = pd.DataFrame({
        'city': ['Moscow', 'SPb', 'Kazan', 'Moscow', 'SPb', 'Novosibirsk', 
                 'Moscow', 'Kazan', 'SPb', 'Moscow']
    })
    sample_y = np.array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1])
    
    te = TargetEncoder(smooth='auto', random_state=RANDOM_STATE)
    encoded = te.fit_transform(sample_X, sample_y)
    
    print("Target Encoding:")
    print(pd.DataFrame({
        'city': sample_X['city'],
        'target': sample_y,
        'encoded': encoded.flatten()
    }))\n    
except ImportError:
    print("TargetEncoder доступен в sklearn >= 1.3")
    print("Для более ранних версий используйте category_encoders.TargetEncoder")


---
## 5. Полный пайплайн с кросс-валидацией

Главное преимущество пайплайнов — их можно передать в `cross_val_score`, и всё будет работать корректно!


### Мини-проект: доведите пайплайн до продакшн-формата
Сформулируйте задачу (классификация или регрессия) и соберите пайплайн с тремя слоями: препроцессинг → модель → постобработка.
- Добавьте GridSearchCV/RandomizedSearchCV для подбора гиперпараметров.
- Проверьте устойчивость через кросс-валидацию.
- Сохраните ключевые метрики и короткий вывод.


In [ ]:
# TODO: ваш экспериментальный пайплайн
# from sklearn.model_selection import GridSearchCV
# search = GridSearchCV(full_pipeline, {...}, cv=3, scoring="roc_auc")
# search.fit(X, y)
# print("Best params:", search.best_params_)
# print("Best score:", search.best_score_)


In [ ]:
# Создаём пайплайн заново для чистоты
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Полный пайплайн
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE))
])

# Кросс-валидация — препроцессинг fit'ится на каждом train фолде!
cv_scores = cross_val_score(
    pipeline, df, y, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc'
)

print(f"CV ROC-AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"Scores: {cv_scores}")


**Почему это работает правильно:**

На каждом фолде:
1. `preprocessor.fit()` вызывается только на train части
2. `preprocessor.transform()` применяется к val части
3. Модель обучается на трансформированном train
4. Предсказание делается на трансформированном val

Никаких утечек! 🎉

---
## 6. Практика


In [ ]:
# TODO: Создайте свой пайплайн
# 1. Для числовых признаков: заполните пропуски средним, масштабируйте MinMaxScaler
# 2. Для категориальных: заполните пропуски константой 'missing', OneHotEncoder
# 3. Модель: LogisticRegression
# 4. Оцените качество через cross_val_score

# Используйте тот же датасет df и таргет y

# ВАШ КОД ЗДЕСЬ
my_numeric_transformer = Pipeline([
    # ...
])

my_categorical_transformer = Pipeline([
    # ...
])

my_preprocessor = ColumnTransformer([
    # ...
])

my_pipeline = Pipeline([
    # ...
])

# Проверка
# my_cv_scores = cross_val_score(my_pipeline, df, y, cv=5, scoring='roc_auc')
# print(f"CV ROC-AUC: {my_cv_scores.mean():.4f}")


In [ ]:
# SOLUTION
my_numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

my_categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

my_preprocessor = ColumnTransformer([
    ('num', my_numeric_transformer, numeric_features),
    ('cat', my_categorical_transformer, categorical_features)
])

my_pipeline = Pipeline([
    ('preprocessor', my_preprocessor),
    ('classifier', LogisticRegression(random_state=RANDOM_STATE))
])

# Проверка
my_cv_scores = cross_val_score(my_pipeline, df, y, cv=5, scoring='roc_auc')
print(f"CV ROC-AUC: {my_cv_scores.mean():.4f} ± {my_cv_scores.std():.4f}")


---
## Что дальше?

Теперь вы умеете:
- Строить безопасные пайплайны без утечек данных
- Обрабатывать разные типы признаков с ColumnTransformer
- Выбирать подходящий способ кодирования категорий

**Следующий шаг:** `03_visual_diagnostics.ipynb` — визуализация для EDA и диагностики моделей.

**Также рекомендуем:**
- `01_cv_metrics_leakage.ipynb` — кросс-валидация и метрики
- `boostings_101.ipynb` — подбор гиперпараметров (там тоже используются пайплайны!)
